In [38]:
import os
import re
import shutil
from pathlib import Path

DEPENDENCIES_SAVE_PATH = Path('dependencies')
SOURCE_CODE_SAVE_PATH = Path('source-code')
SOURCE_CODE_PATH = Path('../')
#pip install -r C://Users//Tolga//Desktop//EPOCH-IV//q3-harmful-brain-activity//submission//dependencies//requirements.txt --no-index --find-links=file://C://Users//Tolga//Desktop//EPOCH-IV//q3-harmful-brain-activity//submission//dependencies//tmp
UPDATE_DEPENDENCIES = False
UPDATE_SOURCE_CODE = True

# Manages Depenedencies

### Download Dependencies and ZIP them

In [39]:
if UPDATE_DEPENDENCIES:
    if os.path.exists(DEPENDENCIES_SAVE_PATH):
        print('Cleaning the dependencies folder')
        for filename in os.listdir(DEPENDENCIES_SAVE_PATH):
            file_path = os.path.join(DEPENDENCIES_SAVE_PATH, filename)
            if filename != 'tmp':
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
    else:
        os.makedirs(DEPENDENCIES_SAVE_PATH)

    print('Copying the requirements.txt file and excluding -e')
    with open(SOURCE_CODE_PATH / 'requirements.txt', 'r') as f:
        lines = f.readlines()
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'w') as f:
        for line in lines:
            if line.startswith('-e'):
                continue
            f.write(line)

    if not os.path.exists(DEPENDENCIES_SAVE_PATH / 'tmp'):
        os.makedirs(DEPENDENCIES_SAVE_PATH / 'tmp')
    print('Downloading the dependencies')
    !pip download -r {DEPENDENCIES_SAVE_PATH / 'requirements.txt'} -d {DEPENDENCIES_SAVE_PATH / 'tmp'}

    print('Zipping the downloaded dependencies')
    shutil.make_archive(DEPENDENCIES_SAVE_PATH / 'dependencies', 'zip', DEPENDENCIES_SAVE_PATH / 'tmp')
    shutil.move(DEPENDENCIES_SAVE_PATH / 'dependencies.zip', DEPENDENCIES_SAVE_PATH / 'dependencies.no_unzip')
    if os.path.exists(DEPENDENCIES_SAVE_PATH / 'tmp'):
        shutil.rmtree(DEPENDENCIES_SAVE_PATH / 'tmp')

    print('Copying the dataset-metadata.json file')
    shutil.copy('dataset-metadata-dependencies.json', DEPENDENCIES_SAVE_PATH / 'dataset-metadata.json')

    print('Excluding --find-files in requirements.txt')
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'r') as f:
        lines = f.readlines()
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'w') as f:
        for line in lines:
            if line.startswith('--find-links'):
                continue
            f.write(line)

    print('Done')

Cleaning the dependencies folder
Copying the requirements.txt file and excluding -e
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 157.6 MB/s eta 0:00:16
     ---------------------------------------- 0.0/2.5 GB 121.6 MB/s eta 0:00:21
     ---------------------------------------- 0.0/2.5 GB 108.8 MB/s eta 0:00:23
     ---------------------------------------- 0.0/2.5 GB 131.2 MB/s eta 0:00:19
     ---------------------------------------- 0.0/2.5 GB 131.2 MB/s eta 0:00:19
      --------------------------------------- 0.0/2.5 GB 129.5 MB/s eta 0:00:19
      --------------------------------------- 0.0/2.5 GB 110.0 MB/s eta 0:00:22
      --------------------------------------- 0.0/2.5 GB 110.0 MB/s eta 0:00:22
      --------------------------------------- 0.0/2.5 GB 129.5 MB/s eta 0:00:19
      --------------------------------------- 0.1/2.5 GB

### Upload Dependencies to Kaggle as a Dataset

In [40]:
if UPDATE_DEPENDENCIES:
    # !kaggle datasets create -p ./dependencies
    !kaggle datasets version -p ./dependencies -m "Update Dependencies"


Starting upload for file dependencies.no_unzip
Upload successful: dependencies.no_unzip (2GB)
Starting upload for file requirements.txt
Upload successful: requirements.txt (340B)
Skipping folder: tmp; use '--dir-mode' to upload folders
Dataset version is being created. Please check progress at https://www.kaggle.com/justanotherariel/epoch-hms-dependencies



  0%|          | 0.00/2.41G [00:00<?, ?B/s]
  0%|          | 16.0k/2.41G [00:00<11:53:25, 60.5kB/s]
  0%|          | 336k/2.41G [00:00<1:05:11, 662kB/s]   
  0%|          | 400k/2.41G [00:00<1:17:52, 554kB/s]
  0%|          | 2.14M/2.41G [00:00<11:34, 3.72MB/s]
  0%|          | 2.59M/2.41G [00:01<16:52, 2.55MB/s]
  0%|          | 7.27M/2.41G [00:01<04:19, 9.95MB/s]
  0%|          | 9.03M/2.41G [00:01<04:18, 9.98MB/s]
  1%|          | 13.1M/2.41G [00:01<02:40, 16.0MB/s]
  1%|          | 17.9M/2.41G [00:01<01:56, 22.1MB/s]
  1%|          | 23.9M/2.41G [00:01<01:21, 31.4MB/s]
  1%|          | 28.1M/2.41G [00:02<01:14, 34.2MB/s]
  1%|▏         | 32.8M/2.41G [00:02<01:07, 37.9MB/s]
  1%|▏         | 36.9M/2.41G [00:02<01:07, 37.6MB/s]
  2%|▏         | 41.7M/2.41G [00:02<01:02, 40.6MB/s]
  2%|▏         | 46.2M/2.41G [00:02<00:59, 42.5MB/s]
  2%|▏         | 51.6M/2.41G [00:02<00:54, 46.4MB/s]
  2%|▏         | 56.2M/2.41G [00:02<00:56, 44.7MB/s]
  2%|▏         | 60.6M/2.41G [00:02<00:56, 44.7M

# Manage Source Code

### Copy Source Code and ZIP it

In [41]:
if UPDATE_SOURCE_CODE:
    if os.path.exists(SOURCE_CODE_SAVE_PATH):
        shutil.rmtree(SOURCE_CODE_SAVE_PATH)
    os.mkdir(SOURCE_CODE_SAVE_PATH)

    # Copy Source Code to submission/source_code
    relevant_files = ['src/', 'conf/', 'tm/', 'submit.py']
    exluded_files = ['__pycache__']
    for file in relevant_files:
        if os.path.isdir(SOURCE_CODE_PATH / file):
            # Copy directory, skip excluded files with shutil
            shutil.copytree(SOURCE_CODE_PATH / file, SOURCE_CODE_SAVE_PATH / "tmp" / file, ignore=shutil.ignore_patterns(*exluded_files))
        else:
            # Copy file
            shutil.copy(SOURCE_CODE_PATH / file, SOURCE_CODE_SAVE_PATH / "tmp" / file)

    # Zip source_code
    shutil.make_archive(SOURCE_CODE_SAVE_PATH / 'source-code', 'zip', SOURCE_CODE_SAVE_PATH / "tmp")
    shutil.rmtree(SOURCE_CODE_SAVE_PATH / "tmp")

    # Copy dataset-metadata.json to submission
    shutil.copy('dataset-metadata-source-code.json', SOURCE_CODE_SAVE_PATH / 'dataset-metadata.json')

    print('Submission files saved to source_code')

Submission files saved to source_code


### Upload Source Code

In [42]:
if UPDATE_SOURCE_CODE:
    # !kaggle datasets create -p ./source-code
    !kaggle datasets version -p ./source-code -m "Update Source Code"

Starting upload for file source-code.zip
Upload successful: source-code.zip (333MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/justanotherariel/epoch-hms-source-code



  0%|          | 0.00/333M [00:00<?, ?B/s]
  0%|          | 16.0k/333M [00:00<1:41:41, 57.3kB/s]
  0%|          | 384k/333M [00:00<05:42, 1.02MB/s]   
  1%|          | 2.64M/333M [00:00<00:56, 6.08MB/s]
  1%|          | 3.33M/333M [00:01<01:33, 3.71MB/s]
  2%|▏         | 6.95M/333M [00:01<00:35, 9.54MB/s]
  3%|▎         | 8.56M/333M [00:01<00:32, 10.4MB/s]
  5%|▌         | 17.0M/333M [00:01<00:12, 26.7MB/s]
  6%|▌         | 20.8M/333M [00:01<00:11, 28.1MB/s]
  7%|▋         | 24.7M/333M [00:01<00:10, 30.6MB/s]
  9%|▊         | 29.0M/333M [00:01<00:09, 34.1MB/s]
 10%|█         | 33.7M/333M [00:01<00:08, 37.6MB/s]
 12%|█▏        | 38.4M/333M [00:01<00:07, 40.9MB/s]
 13%|█▎        | 43.2M/333M [00:01<00:07, 43.0MB/s]
 14%|█▍        | 47.7M/333M [00:02<00:06, 44.1MB/s]
 16%|█▌        | 52.1M/333M [00:02<00:06, 43.7MB/s]
 17%|█▋        | 56.7M/333M [00:02<00:06, 44.5MB/s]
 18%|█▊        | 61.2M/333M [00:02<00:06, 45.1MB/s]
 20%|█▉        | 66.3M/333M [00:02<00:05, 47.3MB/s]
 21%|██▏       |